trying to map the GLS approach of https://github.com/kundajelab/coessentiality to our dataset: 

In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
version = 'old'
# version = 'clare_basis'
# version = 'clare_all'


In [4]:
if version == 'old':
    fn = 'result_logfc_matrix_2020_08_27.csv' ## data set without Claire's screens
elif version == 'clare_all':
    fn = 'result_logfc_matrix_2021_08_27.csv' ## data set with all of Claire's pairwise combinations
elif version == 'clare_basis':
    fn = 'result_logfc_matrix_2021_08_27_BASIS.csv' ## data set only Claire's mutant vs. wildtype screens

path = '../data/standardized_data/'
df_lfc = pd.read_csv(os.path.join(path, fn), index_col = 0)
df_lfc.shape

(4055, 64)

In [5]:
df_lfc = df_lfc.dropna(axis=0)
df_lfc.shape

(3971, 64)

In [6]:
df_lfc.head(2)

,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,Rv3811_KO_vs_Rubin_FLUTE_WT,...,zhang_AA_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_control,zhang_Fe_1.5mM_vs_zhang_Fe_450uM,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45,zhang_wt_mouse_d10_vs_zhang_input_library,zhang_wt_mouse_d45_vs_zhang_input_library
Rv_ID,,,,,,,,,,,,,,,,,,,,,
Rv0001,-0.41,0.01,0.0,0.0,0.0,3.12,-0.06,0.0,-0.06,1.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Rv0002,3.28,2.38,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Remove cell lines with any missing genes


In [7]:
df_lfc.dropna(axis=1, inplace=True)
df_lfc.shape

(3971, 64)

### Warp screen data and intercept based on covariance of screens


We have a problem: the covariance matrix is singular...  

In [8]:
np.linalg.inv(np.cov(df_lfc.values.T))

array([[ 5.75279138, -3.78455995, -0.01909157, ...,  0.08825097,
         0.11636329,  0.14638533],
       [-3.78455995,  6.36622702, -0.01534854, ...,  0.02089874,
        -0.02953769, -0.01429268],
       [-0.01909157, -0.01534854,  4.89343357, ..., -0.03478133,
        -0.0817099 , -0.10535316],
       ...,
       [ 0.08825097,  0.02089874, -0.03478133, ...,  0.87887311,
        -0.1584684 ,  0.34753166],
       [ 0.11636329, -0.02953769, -0.0817099 , ..., -0.1584684 ,
         1.33156975, -0.29142479],
       [ 0.14638533, -0.01429268, -0.10535316, ...,  0.34753166,
        -0.29142479,  0.69151514]])

for now i'm taking the pseudoinverse: 

In [64]:
cov = np.cov(df_lfc.values.T)
pinv_mat = np.linalg.pinv(cov)


In [65]:
print(np.linalg.eigvalsh(cov)[:10])

[0.00947857 0.0173344  0.02543065 0.03167455 0.04938763 0.08435533
 0.09601268 0.10153187 0.10389432 0.10633217]


In [66]:
print(np.linalg.eigvalsh(pinv_mat)[:10])


[0.05116849 0.11162252 0.1304997  0.1374336  0.14768238 0.18190019
 0.19356684 0.22882941 0.24084519 0.26679639]


But this leads to a new problem: the pseudo-inverse is not positive definite: 

In [11]:
cholsigmainv = np.linalg.cholesky(np.linalg.inv(np.cov(df_lfc.values.T)))
warped_screens = df_lfc.values @ cholsigmainv
warped_intercept = cholsigmainv.sum(axis=0)

## In case it does work, you move on to this: 

In [12]:
def linear_regression(warped_screens, warped_intercept):
    GLS_coef = np.empty((len(warped_screens), len(warped_screens)))
    GLS_se = np.empty((len(warped_screens), len(warped_screens)))
    ys = warped_screens.T
    for gene_index in range(len(warped_screens)):
        X = np.stack((warped_intercept, warped_screens[gene_index]), axis=1)
        coef, residues = np.linalg.lstsq(X, ys, rcond=None)[:2]
        df = warped_screens.shape[1] - 2
        GLS_coef[gene_index] = coef[1]
        try:
            GLS_se[gene_index] = \
                np.sqrt(np.linalg.pinv(X.T @ X)[1, 1] * residues / df)
        except:
            print(gene_index) 
    return GLS_coef, GLS_se

In [13]:
GLS_coef, GLS_se = linear_regression(warped_screens, warped_intercept)


416
661
675
1327
1908
1914
2429
2522
2625
2686
2696
3045
3654
3970


In [14]:
ys = warped_screens.T
gene_index = 416
X = np.stack((warped_intercept, warped_screens[gene_index]), axis=1)
coef, residues = np.linalg.lstsq(X, ys, rcond=None)[:2]
df = warped_screens.shape[1] - 2

In [15]:
coef[1]

array([0., 0., 0., ..., 0., 0., 0.])